## Overview 

Author: Chenyu Shen (Rainy)  
Last Update: August 16, 2020

This notebook is an automation of data mining from wrds libary. For the purpose of this research, we mine data from Compustat dataset. The data result changes daily, and we can change the variable names in the code for different analysis.

## Set up

In [1]:
# Set up standard python analysis framework
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
# Set up wrds database connection
import wrds
username = "yourusername" # enter username here, enter password when prompted
db = wrds.Connection(wrds_username=username)

Loading library list...
Done


In [ ]:
# sets up a "pgpass" file that do not need to re-enter the username and password everytime
# once done once, you don't have to run this cell in future
db.create_pgpass_file() 

In [3]:
# Test database connection
db.close()
db = wrds.Connection(wrds_username=username)

## Data Inspection 
Before data mining, inspect certain features of the wrds libarary and the dataset we are interested in.

In [3]:
# List libraries in wrds
db.list_libraries()[:5]

['wrds_lib_internal', 'compg', 'ppubsamp_d4d_new', 'ktmine', 'compsamp']

In [4]:
# List tables for specified library
# E.g. compustat : 'comp'
db.list_tables(library = 'comp')[0]

'filings'

In [5]:
# Describe selected. E.g. Compustat database - fundamentals annual daily table(North America) 
col_headers = db.describe_table(library = 'comp', table = 'funda')
# compd: Compustat - NA, Bank, Global - Daily Update

Approximately 820642 rows in comp.funda.


In [6]:
# Check variables of this table
all_var = col_headers['name'].to_list()
print("Number of variables in this table: ", len(all_var))
print(all_var[:50]) # list of all variables in selected table

Number of variables in this table:  948
['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'cusip', 'conm', 'acctchg', 'acctstd', 'acqmeth', 'adrr', 'ajex', 'ajp', 'bspr', 'compst', 'curcd', 'curncd', 'currtr', 'curuscn', 'final', 'fyr', 'ismod', 'ltcm', 'ogm', 'pddur', 'scf', 'src', 'stalt', 'udpl', 'upd', 'apdedate', 'fdate', 'pdate', 'acchg', 'acco', 'accrt', 'acdo', 'aco', 'acodo', 'acominc', 'acox', 'acoxar', 'acqao', 'acqcshi', 'acqgdwl', 'acqic', 'acqintan']


In [7]:
# number of companies with filings in 2019
# this number keeps updating in real time
db.raw_sql(
"""
SELECT  COUNT(DISTINCT gvkey)
FROM comp.funda
WHERE fyear >= 2019
"""
)

,count
0,10229


In [8]:
# number of companies with filings in 2019, quarterly database
# this number keeps updating in real time
db.raw_sql(
"""
SELECT  COUNT(DISTINCT gvkey)
FROM comp.fundq
WHERE fyearq >= 2019
"""
)

,count
0,11394


## Data Mining 
Get data from Compustat with selected table and variables. Refer to Compustat documentation on wrds to see the table name, and variable codes [WRDS datasets reference](https://wrds-web.wharton.upenn.edu/wrds/tools/variable.cfm?library_id=162).

In [8]:
%%time
# query with sql
# from table: North America Fundamentals Annual (Daily)
df_na_annual = db.raw_sql(
"""
SELECT  gvkey,
        cik,
        cusip,
        conm,
        fyear,
        naicsh,
        revt AS "Revenue - Total",
        tie AS "Interest Expense Total (Financial Services)",
        txt AS "Income Taxes - Total",
        xopr AS "Operating Expenses Total",
        xt AS "Expense - Total",
        ni AS "Net Income(Loss)",
        ch AS "Cash",
        at AS "Assets - Total",
        lt AS "Liabilities - Total",
        seq AS "Stockholders' Equity - Total",
        reuna AS "Retained Earnings Unadjusted",
        reuna - seq AS "Contributed Capital",
        dvt AS "Dividends - Total",
        at - ch AS "NCAssets",
        at - ch - lt AS "NetNCAssets",
        ni/NULLIF(at,0) AS "ROA",
        ni/NULLIF(seq,0) AS "ROE"
FROM comp.funda
WHERE (fyear >= 2013 AND fyear <= 2019) AND
      fdate IS NOT NULL AND
      naicsh IS NOT NULL AND
      ni IS NOT NULL AND
      at IS NOT NULL AND
      seq IS NOT NULL 
"""
)


CPU times: user 433 ms, sys: 224 ms, total: 657 ms
Wall time: 44 s


In [9]:
def describe_data(df):
    print("number of records ", len(df))
    print("number of companies ", len(np.unique(df["conm"])))

In [10]:
# Describe data
describe_data(df_na_annual)

number of records  41442
number of companies  9220


In [34]:
# drop duplicates
df_na_annual.drop_duplicates(inplace = True)
# drop null value
df_na_annual.dropna(inplace = True)

In [35]:
describe_data(df_na_annual) # data result after previous processing

number of records  42985
number of companies  8470


In [36]:
# store data result to csv file
df_na_annual.to_csv("data_output/df_na_annual_13-19.csv",index = False)

## Reference:  
1) WRDS's introduction site of programmin using python:  
https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/python-from-your-computer/  
2) WRDS Python package documentation:  
https://pypi.org/project/wrds/  
3) WRDS datasets reference:  
https://wrds-web.wharton.upenn.edu/wrds/tools/variable.cfm?library_id=162
